In [ ]:
#%matplotlib widget
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sector_expts2 as se
import xarray as xr
import string
import cartopy.crs as ccrs
import MITgcmutils as mu
import matplotlib.colors as colors
import matplotlib.pylab as pl
import string
from scipy.stats import pearsonr, spearmanr
from scipy.constants import golden
import importlib
import os,sys
from IPython.display import HTML
importlib.reload(se)
import warnings
warnings.filterwarnings('ignore')
from scipy.signal import savgol_filter
from scipy.special import legendre
from scipy.interpolate import interp1d
def cb_good(cb):
    cb.formatter.set_powerlimits((0,0))
    cb.update_ticks()
    return cb

### _CASENAME_

Q = xxmW top, xxmW bottom

with xxpsu mean ocean salinity

viscosity: diffusivity:

IC: Bottom is xxmK warmer than the top

In [ ]:
expt=se.Experiment(name='_CASENAME_')

In [ ]:
print('E='+str(expt.E))
print('Ra='+str(expt.ra().values))
print('RaE4/3='+str(expt.ra().values*expt.E**(4/3)))
Ra_RaT=0.1*expt.ra()*expt.E**(3/2)
print('Ra_RaT='+str(Ra_RaT.values))
print('Rastar_q='+str(expt.rastarq))
      
print('Ro*='+str(expt.rostar))
print('Ro_loc='+str(expt.roloc().values))

print('l_rot='+str(expt.lrot))
print('u_rot='+str(expt.urot))
print('lu_rot='+str(expt.urot*expt.lrot))
print('l_cone='+str(expt.lcone))
print('u_cone='+str(expt.ucone))
print('lu_cone='+str(expt.ucone*expt.lcone))

In [ ]:
_,_,_=expt.monitor(iteration=np.NaN,pltarray=(1,1),pltvar=['q'],flip=False,dimmeths={'lon':'mean'},figsize=(12,6),wdirs=[1])

In [ ]:
_,_,_=expt.monitor(iteration=np.NaN,pltarray=(1,1),pltvar=['q'],flip=False,dimmeths={'lon':'mean','lat':'mean'},figsize=(12,6),wdirs=[1])

In [ ]:
_,_,_=expt.monitor(iteration=np.NaN,pltarray=(1,1),pltvar=['T'],flip=False,dimmeths={'lon':'mean','z':58},ylims=[50,80],figsize=(12,6))

In [ ]:
ft,axt,dt=expt.monitor(iteration=np.Inf,dimmeths={'Time':0,'lon':'mean'})

In [ ]:
ft,axt,dt=expt.monitor(runpathstr="'./'",iteration=np.Inf,pltarray=(4,1),
    pltvar=['T','S','W','U'],wdirs=[0,0,1,1],dimmeths={'Time':0,'lon':'mean'},
    figsize=(6,18), 
    pltcontour={'U':[0]},
    cms={'T':'RdBu_r','S':'BrBG_r'},vminmax={'T':[-0.00,0.05],'S':[-6e-3,6e-3],'U':[-8e-4,8e-4]})
axt[0].set_ylim([-expt.Htot/1e3+expt.drf[-1]/1e3+expt.Hice0/1e3*(not expt.topo),-expt.Hice0/1e3*(not expt.topo)+expt.Hice0/1e3*(not expt.topo)])


In [ ]:
f,ax,d=expt.monitor(iteration=np.Inf,pltarray=(1,3),pltvar=['T','W','U'],wdirs=[1,1,1]
                    ,dimmeths={'z':20,'Time':0,'lon':'anom'},figsize=(12,12),
                    labelorientation='vertical',labelaspect=30,labelpad=0.15,labelshrink=0.8)

In [ ]:
f,ax,d=expt.monitor(iteration=np.Inf,pltarray=(1,3),
      pltvar=['T','W','U'],wdirs=[1,1,1],dimmeths={'lat':0.,'Time':0,'lon':'anom'},vminmax={'T':[-4e-4,4e-4]},
    figsize=(14,12),labelorientation='vertical',labelaspect=30,labelpad=0.15,labelshrink=0.8)

In [ ]:
with plt.style.context(('labelsize15')):
    fig, ax = plt.subplots(1,1, figsize=(4,12))
    
    xskip, zskip = 1,2
    
    xfull=d['T'].lon.values
    zfull=d['T'].z.values/1000+expt.Hice0/1e3
    x = xfull[::xskip]
    z = zfull[::zskip]
    crng=abs(d['T']).max()/1.2
    im = ax.pcolormesh(zfull,xfull,d['T'].values.transpose(),cmap='RdBu_r',vmin=-crng,vmax=crng)
    cb = fig.colorbar(im,shrink=0.8,aspect=30,pad=0.12,orientation='vertical')
    cb.formatter.set_powerlimits((0,0))
    cb.update_ticks()
    ax.set_title('T (shading) U,W (quiver)')
    Q=ax.quiver(z,x, d['W'].values.squeeze()[::zskip,::xskip].transpose(),
                   d['U'].values.squeeze()[::zskip,::xskip].transpose(),scale=0.005,width=0.005)
    ax.set_xlim([zfull[-1],0])
    ax.set_yticks(np.arange(0,21,5))
    
    ax.set_xlabel('z (km)')
    ax.set_ylabel('Lon ($^{\circ}$)')
    ax.quiverkey(Q, 1.24, 0.05, 1e-3, 
             r'1 mm/s',fontproperties={'size':14}) 

In [ ]:
_,runp=expt.get_runpath(expt.itermax)
runp='./'
iters=mu.mds.scanforfiles(expt.path+runp+"W")
iters=iters[1:-1:3]

with plt.style.context(('labelsize15')):
    fig, ax = plt.subplots(2,1, figsize=(5,9),gridspec_kw={'height_ratios': [1,3]},sharex=True)
    
    xskip, zskip = 20,15
    d=expt.get(['T','S','W','V','q'],runpathstr="''",iteration=np.Inf,dimmeths={'lon':0})
    imq=[ax[0].plot(d['q'].lat,d['q'].squeeze(),color='k',lw=2)]
    ax[0].set_ylabel('q (km/My)')
    ax[0].set_title('day {}'.format(np.int(d['q'].Time.squeeze().values)))
    d['T']=d['T']-expt.Tref
    d['S']=d['S']-expt.Sref
    xfull=d['T'].lat.values
    zfull=d['T'].z.values/1000
    d['T']['z']=zfull
    d['S']['z']=zfull
    x = xfull[::xskip]
    z = zfull[::zskip]
    imT=d['T'].plot(add_colorbar=False, add_labels=False, ax=ax[1],cmap='RdBu_r',vmin=-1e-2,vmax=3e-2)
    #imS=d['S'].plot.contour(add_colorbar=False, add_labels=False, ax=ax[1],levels=[-1e-3,-1e-4,1e-4,1e-3],colors='k',zorder=0)
    imS=[ax[1].contour(xfull,zfull,d['S'].values.squeeze(),colors='k',levels=[-1e-3,-1e-4,1e-4,1e-3])]
    cb = fig.colorbar(imT,shrink=0.8,aspect=30,pad=0.16,orientation='horizontal')
    cb.formatter.set_powerlimits((0,0))
    cb.update_ticks()
    ax[1].set_title('T (shading) S (contour) V,W (quiver)')
    Q=ax[1].quiver(x,z, d['V'].values.squeeze()[::zskip,::xskip],
                   d['W'].values.squeeze()[::zskip,::xskip],scale=0.05)
    ax[1].set_ylim([zfull[-1],0])
    ax[1].set_xticks(np.arange(0,x[-1]+0.1,5000))
    ax[1].set_xticklabels(np.arange(0,x[-1]+0.1,5000)/1000)
    
    ax[1].set_xlabel('y (km)')
    ax[1].set_ylabel('z (km)')
    ax[1].quiverkey(Q, 0., -0.25, 3e-3, 
             r'3 mm/s',fontproperties={'size':14}) 
    
    def animate(i):
        blockPrint()
        d=expt.get(['T','S','W','V','q'],runpathstr="''",iteration=iters[i],dimmeths={'lon':0})
        for tp in ax[0].lines+ax[0].collections:
            tp.remove()
        imq[0]=[ax[0].plot(d['q'].lat,d['q'].squeeze(),color='k',lw=2)]
        ax[0].set_title('day {}'.format(np.int(d['q'].Time.squeeze().values)))
        d['T']=d['T']-expt.Tref
        d['S']=d['S']-expt.Sref
        d['T']['z']=zfull
        d['S']['z']=zfull
        imT.set_array(d['T'].values.ravel(order='C'))
        for tp in imS[0].collections:
            tp.remove()
        imS[0]=ax[1].contour(xfull,zfull,d['S'].values.squeeze(),colors='k',levels=[-3e-2,-1e-2,-1e-3,-3e-4,3e-4,1e-3,1e-2,3e-2])
        Q.set_UVC(d['V'].values.squeeze()[::zskip,::xskip],
                 d['W'].values.squeeze()[::zskip,::xskip])
        return imT,Q
    def blockPrint():
        sys.stdout = open(os.devnull, 'w')
    
    ani=animation.FuncAnimation(fig, animate, frames=len(iters), blit=True)

In [ ]:
HTML(ani.to_jshtml())


In [ ]:
HTML(ani.to_html5_video())

In [ ]:
_,ax,d=expt.monitor(runpathstr="'/'",iteration=np.Inf,sample=1,pltarray=(3,1),pltvar=['q','bulk_heatflux_y','heatflux_surf'],dimmeths={'Time':'mean','lon':'mean'},figsize=(5,10),sharey=False,wdirs=[1,0,0])
adv=expt.bulk_heatflux_y_adv(runpath='/',iteration=np.Inf,sample=1)
ax[1].plot(adv.lat[::10]/1e3,adv[::10],'r--^',linewidth=1)
dif=expt.bulk_heatflux_y_dif(runpath='/',iteration=np.Inf,sample=1)
ax[1].plot(dif.lat[::10]/1e3,dif[::10],'b--o',linewidth=1)
ax[1].legend(['total','advective','diffusive'])
ax[1].plot(adv.lat,adv.lat*0,'k--')
#bndthickness=expt.hfacc.where(expt.hfacc!=1,0).max(dim='z').squeeze()
#ax[2].plot(bndthickness.lat,bndthickness-0.5,'b--')

In [ ]:
Hy=expt.bulk_heatflux_y(runpath='',iteration=np.NaN,sample=1)
Hy_smt=savgol_filter(Hy[1:], 13, 1,axis=0).squeeze()
Hy=Hy[1:].values

Hconv=-1e6*(Hy[1:]-Hy[:-1])/expt.dyc[1:-1,0].squeeze()+expt.Q[1:-1]
Hconv=-1e6*(Hy_smt[1:]-Hy_smt[:-1])/expt.dyc[1:-1,0].squeeze()+expt.Q[1:-1]
H_surf=expt.heatflux_surf(runpath='',iteration=np.NaN,sample=1).mean('lon').mean('Time').squeeze()
q=expt.q(runpath='',iteration=np.NaN,sample=1,inmassunit=True).mean('lon').squeeze().values
Hlatent=expt.lf*q


with plt.style.context(('labelsize15')):
    f,ax=plt.subplots(1,1,figsize=[5,4],sharex=True)
    
    ax.plot(H_surf.lat[1:-1]/1e3,Hconv,'r-')
    ax.plot(H_surf.lat[1:-1]/1e3,H_surf[1:-1],'k--')
    ax.set_title('Ice-ocean heat exchange (W/m$^2$)')
    ax.set_xlabel('y (km)')

In [ ]:
varlist=['T','S','Rho','U','V','W','Psi']
d=expt.get(varlist,runnum=np.Inf,iteration=np.NaN,sample=3)

In [ ]:
fig,axf,dmeanxt,figname=se.my2dplt(expt,d,(3,2),pltvar=['T','S','Rho','U','W','Psi'],
            dimmeths={'Time':'mean','lon':'mean'},flip=True,wdirs=[0,0,0,1,1,1],
            figsize=(12,12),projection=None,pltdir='F',
            savefig=0,labelorientation=None,xlabelpad=0,ylabelpad=20)

In [ ]:
_,_,_,_=se.my2dplt(expt,d,pltarray=(1,2),pltvar=['q','bulk_heatflux_y'],dimmeths={'Time':0,'lon':'mean'},figsize=(10,3.4),sharey=False,wdirs=[1,1])

In [ ]:
fig,axf,dmeanxt,figname=se.my2dplt(expt,d,(3,2),pltvar=['T','S','Rho','U','W','Psi'],
            dimmeths={'z':30,'lon':'mean'},flip=True,wdirs=[0,0,0,1,1,1],
            figsize=(12,12),projection=None,pltdir='F',
            savefig=0,labelorientation=None,xlabelpad=0,ylabelpad=20)

In [ ]:
fig,axf,dmeanxt,figname=se.my2dplt(expt,d,(1,3),pltvar=['T','U','W'],
            dimmeths={'z':7,'Time':np.Inf},flip=True,wdirs=[0,1,1],
            figsize=(12,12),projection='sphere',pltdir='F',
            savefig=0,labelorientation=None,xlabelpad=0,ylabelpad=20)

In [ ]:
varlist=['T','S','Rho','U','V','W','Psi','q','bulk_heatflux_y']
d0=expt.get(varlist,runnum=0,iteration=2160,sample=3)

In [ ]:
fig,axf,dmeanxt0,figname=se.my2dplt(expt,d0,(3,2),pltvar=['T','S','Rho','U','W','Psi'],
            dimmeths={'Time':0,'lon':'mean'},flip=True,wdirs=[0,0,0,1,1,1],
            figsize=(12,12),projection=None,pltdir='F',
            savefig=0,labelorientation=None,xlabelpad=0,ylabelpad=20)

In [ ]:
_,_,_,_=se.my2dplt(expt,d0,pltarray=(1,2),pltvar=['q','bulk_heatflux_y'],dimmeths={'Time':0,'lon':'mean'},figsize=(10,3.4),sharey=False,wdirs=[1,1])

In [ ]:
fig,axf,dmeanxt0,figname=se.my2dplt(expt,d0,(3,2),pltvar=['T','S','Rho','U','W','Psi'],
            dimmeths={'Time':100,'lon':'mean'},flip=True,wdirs=[0,0,0,1,1,1],
            figsize=(12,12),projection=None,pltdir='F',
            savefig=0,labelorientation=None,xlabelpad=0,ylabelpad=20)

In [ ]:
fig,axf,dmeanxt0,figname=se.my2dplt(expt,d0,(3,2),pltvar=['T','S','Rho','U','W','Psi'],
            dimmeths={'Time':np.Inf,'lon':'mean'},flip=True,wdirs=[0,0,0,1,1,1],
            figsize=(12,12),projection=None,pltdir='F',
            savefig=0,labelorientation=None,xlabelpad=0,ylabelpad=20)